In [ ]:
pip install huggingface_hub transformers Langchain

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from collections import Counter
import matplotlib.pyplot as plt

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Initialize sentiment counters
sentiment_counters = Counter()

print("Welcome to the GPT-3 Conversation! Type 'exit' to quit.")

chat_history_ids = None

while True:
    new_user_input = input(">> User: ")
    if new_user_input.lower() == "exit":
        break

    new_user_input_ids = tokenizer.encode(new_user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    generated_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Analyze sentiment of the generated response
    sentiment_result = sentiment_pipeline(generated_response)

    # Update sentiment counters
    sentiment_counters.update([sentiment_result[0]['label']])

    # Print the generated response and its sentiment
    print(f"DialoGPT: {generated_response}")
    print(f"Sentiment: {sentiment_result[0]['label']} (Score: {sentiment_result[0]['score']})")

# Generate sentiment analysis report
print("\n===== Sentiment Analysis Report =====")
total_responses = sum(sentiment_counters.values())
print(f"Total responses: {total_responses}")

for sentiment, count in sentiment_counters.items():
    percentage = (count / total_responses) * 100
    print(f"{sentiment.capitalize()} sentiment: {count} ({percentage:.2f}%)")

# Generate pie chart visualization
labels = sentiment_counters.keys()
sizes = sentiment_counters.values()

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%')
ax.axis('equal')
ax.set_title('Sentiment Distribution')

plt.show()
